# Kryptologie Lab - Übung 05 - Substitutions Permutations Netwerk
- Designprinzip für Blockchiffren
- Lokale Substitution durch S Boxen
- ’Globale’ Permutation
- Schlüsseladdition
- Arbeitet in Runden
- Beispiel: AES
- ![SPN](SPN.PNG)

In [123]:
from tqdm.notebook import tqdm

In [124]:
sbox = [0xE, 0x4, 0xD, 0x1, 0x2, 0xF, 0xB, 0x8, 0x3, 0xA, 0x6, 0xC, 0x5, 0x9, 0x0, 0x7]
perm = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

In [125]:
# sbox substitution 
def sbox_block(value, inverse=False):
    if inverse:
        return sbox.index(value)
    else: 
        return sbox[value]

# lokale Substitution durch SBox
# sbox auf jeden der 4 Blocke (mit jeweils 4 Bits)
def executeSbox(value,inverse=False):
    block0 = sbox_block(value & 0xF, inverse)
    block1 = sbox_block((value & 0xF0) >> 4, inverse)
    block2 = sbox_block((value & 0xF00) >> 8, inverse)
    block3 = sbox_block((value & 0xF000) >> 12, inverse)
    return (block3 << 12) | (block2 << 8) | (block1 << 4) | block0

# Globale Permutation der 16 Bit
def permutation(value):
    output = 0
    for i in range(1, 16):
        bit = value & 0x1
        output = output | (bit << (perm[i] - 1))
        value = value >> 1
    return output

def addRoundKey(value, key):
    return value ^ key

In [126]:
# Encrypt via SPN in 4 Runden
# Input: 4 Block á 4 Bit
# Output: 4 Block á 4 Bit
def encrypt(value, key):
    for i in range(3):
        value = addRoundKey(value, key)
        value = executeSbox(value)
        value = permutation(value)
    value = addRoundKey(value, key)
    value = executeSbox(value)
    value = addRoundKey(value, key)
    return value

## Testing

In [127]:
for i in range(0, 16):
    encrypted = sbox_block(i)
    decrypted = sbox_block(encrypted, inverse=True)
    assert i == decrypted

value = 0xABCD
encrypted = executeSbox(value)
decrypted = executeSbox(encrypted, inverse=True)
assert value == decrypted

encrypt(0xABCD, 0xFABE)

50256

## Lineare Kryptonanalyse
Idee: Suche lineare Approximation an S-Boxen

Known-plaintext attack

In [128]:
import random

In [129]:
# returns bit of val at position
def getBit(val, index):
    return (val >> (15-index)) & 0x1

# only second and fourth byte and extends to 4 byte. rest zeros
def expandKey(subkey):
    return ((subkey & 0xF) << 8) ^ ((subkey & 0xF0) >> 4)

def getProbabilityOfKeys(pairs):
    possibleKeys = dict()
    # iterate possible keys
    for subkey in range(0xFF+1):
        key = expandKey(subkey)
        hits = 0
        # iterate text-cipher pairs to find correct part of the key
        for pair in pairs:
            x = pair[0]
            y = pair[1]
            v4 = addRoundKey(y, key)
            u4 = executeSbox(v4, inverse=True)
            approx = getBit(x,4) ^ getBit(x,6) ^ getBit(x,7) ^ getBit(u4,5) ^ getBit(u4,7) ^ getBit(u4,13) ^ getBit(u4,15)
            if approx == 0:
                hits += 1
        possibleKeys[subkey] = hits/len(pairs)
    return possibleKeys

def getBestSubKeyOrder(probabilities):
    for data in probabilities.items():
        probabilities.update({data[0]: abs(data[1] - 0.5)})
    return sorted(probabilities.items(), key=lambda item: item[1])

## Generate Text-Cipher Pairs

In [130]:
key = 0xFFAE
amount = 12000

M = []
for i in tqdm(range(amount)):
    input = random.getrandbits(16)
    M.append((input, encrypt(input, key)))

In [131]:
probabilities = getProbabilityOfKeys(M)
subkeysInOrderWithProbability = getBestSubKeyOrder(probabilities)
subkeysInOrder = [entry[0] for entry in subkeysInOrderWithProbability]
print(subkeysInOrder)

[8, 120, 224, 145, 123, 126, 159, 168, 170, 239, 44, 152, 41, 28, 156, 195, 222, 124, 184, 189, 223, 19, 48, 76, 92, 226, 228, 86, 30, 32, 237, 64, 134, 149, 201, 211, 116, 136, 173, 248, 88, 119, 12, 77, 104, 132, 241, 72, 230, 15, 165, 95, 151, 17, 27, 157, 214, 83, 93, 167, 246, 58, 74, 112, 206, 255, 2, 34, 164, 25, 78, 148, 240, 40, 125, 154, 247, 5, 84, 204, 100, 109, 129, 251, 45, 143, 131, 150, 177, 13, 221, 182, 63, 96, 245, 20, 65, 141, 180, 31, 38, 216, 106, 181, 133, 174, 213, 22, 155, 205, 3, 68, 4, 192, 146, 171, 70, 47, 37, 117, 242, 80, 102, 153, 160, 197, 16, 79, 82, 191, 29, 53, 219, 67, 121, 59, 183, 193, 35, 208, 209, 172, 9, 39, 122, 144, 188, 75, 229, 252, 81, 249, 18, 24, 158, 99, 36, 51, 90, 254, 147, 97, 110, 21, 85, 114, 118, 108, 166, 244, 49, 127, 128, 140, 236, 0, 194, 220, 207, 227, 250, 66, 199, 232, 179, 23, 6, 42, 26, 185, 176, 52, 71, 187, 231, 186, 94, 169, 212, 55, 210, 253, 113, 101, 203, 43, 115, 111, 61, 234, 91, 60, 190, 218, 107, 215, 163, 243, 

## Brutforce in better order

In [132]:
countKeyTries = 0
cracked = False
for key in tqdm(subkeysInOrder):
    countKeyTries += 1
    for brutforceSubkey in range(0xFF+1):
        expandedKey = expandKey(key)
        testKey = ((brutforceSubkey & 0xF) << 4) ^ ((brutforceSubkey & 0xF0) << 8) ^ expandedKey
        cipher = encrypt(M[0][0], testKey)
        if cipher == M[0][1]:
            print("Hit")
            correct = 0
            for m in M:
                cipher = encrypt(m[0], testKey)
                if cipher == m[1]:
                    correct += 1
            if correct == len(M):
                print("Key: ", hex(testKey))
                print("Cracked after ", countKeyTries, " tries")
                cracked = True
                break
    if cracked:
        break       

Hit
Key:  0xffae
Cracked after  10  tries



With more than 8000 samples the brutforce step becomes even faster. With less, the correct key is more at the back.